In [4]:
import pandas as pd

# 파일 경로
path_125 = "./../cohort/cohort_ver125_feature_matrix_repaired.csv"
path_182 = "./../cohort/cohort_ver182_winsor_99p.csv"

# CSV 로드
df125 = pd.read_csv(path_125)
df182 = pd.read_csv(path_182)

df182_unique = df182.drop_duplicates(subset=['subject_id'], keep='first')

# subject_id 기준 병합 (125 기준으로 유지)
df_merged = df125.merge(df182_unique, on='subject_id', how='left')

# 저장
df_merged.to_csv("cohort_101_merged_125_182.csv", index=False)

print(df_merged.shape)
print(df_merged.head())


(14312, 40)
   subject_id  hadm_id_x  age_x  gender_x                    race_x  \
0    13647833   20006266     40         1  ASIAN - SOUTH EAST ASIAN   
1    13647833   20006266     40         1  ASIAN - SOUTH EAST ASIAN   
2    13647833   20006266     40         1  ASIAN - SOUTH EAST ASIAN   
3    13647833   20006266     40         1  ASIAN - SOUTH EAST ASIAN   
4    13647833   20006266     40         1  ASIAN - SOUTH EAST ASIAN   

   prefix_len  time_since_ed  time_since_last  is_night  cum_ecg_cnt  ...  \
0           1            0.0                0         0            0  ...   
1           2            0.0         27504819         1            1  ...   
2           3            0.0              654         0            1  ...   
3           4          626.0              626         0            1  ...   
4           5         1240.0              614         0            1  ...   

   age_y  gender_y                    race_y  arrival_transport_y  cci_score  \
0     40         1

In [6]:
import pandas as pd

# 1. 병합된 파일 로드
path_merged = "./../cohort/cohort_101_merged_125_182.csv"
df = pd.read_csv(path_merged)

# 2. prefix 순서대로 정렬 (필요에 따라 정렬 기준 조정 가능)
df = df.sort_values(["subject_id", "hadm_id", "prefix_len"]).reset_index(drop=True)

# 3. 기존 cum_ 컬럼은 버리고 새로 계산할 준비
for col in ["cum_ecg_cnt", "cum_stemi_cnt", "cum_trop_cnt"]:
    if col in df.columns:
        df.drop(columns=[col], inplace=True)

# 4. 이벤트 플래그 정의
# ↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓
# 이 부분은 네 데이터에 맞게 조건만 조정하면 됨.

# (예시 1) current_event_id가 문자열인 경우
#   - ECG_TAKEN, ECG_STEMI_FLAG, TROP_TAKEN, TROP_POSITIVE 같은 이름일 때
# df["is_ecg"]       = df["current_event_id"].isin(["ECG_TAKEN", "ECG_STEMI_FLAG"]).astype(int)
# df["is_stemi_ecg"] = (df["current_event_id"] == "ECG_STEMI_FLAG").astype(int)
# df["is_trop"]      = df["current_event_id"].isin(["TROP_TAKEN", "TROP_POSITIVE"]).astype(int)

# (예시 2) current_event_id가 숫자 코드이고, 예를 들어
#   1: ED_ARRIVAL, 2: ECG_TAKEN, 3: ECG_STEMI_FLAG, 4: TROP_TAKEN, 5: TROP_POSITIVE, ...
# 라고 가정한다면 (실제 코드에 맞게 숫자만 바꿔주면 됨)
df["is_ecg"]       = df["current_event_id"].isin([2, 3]).astype(int)   # ECG_TAKEN + ECG_STEMI_FLAG
df["is_stemi_ecg"] = df["current_event_id"].eq(3).astype(int)         # ECG_STEMI_FLAG
df["is_trop"]      = df["current_event_id"].isin([4, 5]).astype(int)  # TROP_TAKEN + TROP_POSITIVE

# 만약 따로 이벤트 코드가 없고, stemi_flag / trop_pos_flag만 있다면 예를 들어:
# df["is_ecg"]       = (df["stemi_flag"].notna()).astype(int)  # ECG row에서만 stemi_flag가 세팅되어 있다면
# df["is_stemi_ecg"] = (df["stemi_flag"] == 1).astype(int)
# df["is_trop"]      = (df["trop_pos_flag"].notna()).astype(int)

# ↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑
# 여기까지 조건은 네 데이터 구조에 맞게 골라서/수정해서 사용

# 5. 그룹별 누적 합 (prefix 기반 누적)
group_keys = ["subject_id", "hadm_id"]

df["cum_ecg_cnt"]   = df.groupby(group_keys)["is_ecg"].cumsum()
df["cum_stemi_cnt"] = df.groupby(group_keys)["is_stemi_ecg"].cumsum()
df["cum_trop_cnt"]  = df.groupby(group_keys)["is_trop"].cumsum()

# 6. 임시 플래그 컬럼 제거
df = df.drop(columns=["is_ecg", "is_stemi_ecg", "is_trop"])

# 7. 저장
out_path = "./../cohort/cohort_102_recalc_cum.csv"
df.to_csv(out_path, index=False)

print(df[["subject_id", "hadm_id", "prefix_len",
          "cum_ecg_cnt", "cum_stemi_cnt", "cum_trop_cnt"]].head(30))
print("저장 완료:", out_path)


    subject_id   hadm_id  prefix_len  cum_ecg_cnt  cum_stemi_cnt  cum_trop_cnt
0     10000764  27897940           1            0              0             0
1     10000764  27897940           2            0              0             0
2     10000764  27897940           3            1              1             0
3     10000764  27897940           4            2              1             0
4     10000764  27897940           5            2              1             1
5     10000764  27897940           6            3              1             1
6     10000764  27897940           7            4              1             1
7     10000764  27897940           8            5              1             1
8     10000764  27897940           9            6              1             1
9     10000764  27897940          10            7              1             1
10    10000764  27897940          11            8              1             1
11    10000764  27897940          12            8   